In [1]:
from darts import TimeSeries, concatenate
from darts.utils.callbacks import TFMProgressBar
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
from darts.metrics import mape, r2_score
from darts.datasets import EnergyDataset
from darts import concatenate


: 

In [1]:
import time

from statsmodels.datasets import co2
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from statsmodels.tsa.seasonal import STL
import numpy as np
import copy
from sklearn.preprocessing import StandardScaler

import matplotlib.font_manager as fm
import seaborn as sns

In [2]:
import os
os.environ['OMP_NUM_THREADS'] = '4'

from data1.serialize import SerializerSettings
from data1.small_context import get_datasets,get_memorization_datasets

In [3]:
import openai

from models.utils import grid_iter
from models.promptcast import get_promptcast_predictions_data
from models.darts import get_arima_predictions_data
from models.validation_likelihood_tuning import get_autotuned_predictions_data
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from models import llmtime
from models.llmtime import get_llmtime_predictions_data

In [4]:
import pickle

In [5]:
from investigation.utils_others import *

In [6]:
openai.api_key = ""
openai.api_base = os.environ.get("OPENAI_API_BASE", "https://api.openai.com/v1")

api_key=""  # 放入gemini api

In [7]:
class Stl_decomposition:
    def __init__(self, data, period=None, whether_plot=False):
        self.train_data = data[0]
        self.test_data = data[1]
        self.period = period
        self.whether_plot = whether_plot
    
    def stl_decoposition(self):
        '''
        Params:
            data: one-dimension, tuple
            period: often choose the number of days/ months/ years, seems like it can be inferred automatically
        Returns: 
            seas_data: seasonal daata
            trend_data: trend data
            resid_data: residual data
        '''
        res = STL(self.train_data, period=self.period).fit()
        if self.whether_plot:
            res.plot()
        seas_data = res.seasonal
        trend_data = res.trend
        resid_data = res.resid
        return seas_data, trend_data, resid_data
    
    def strength(self):
        seas_data, trend_data, resid_data = self.stl_decoposition()
        resid_var = np.var(resid_data, ddof = 1)
        trend_resid_var = np.var(resid_data+trend_data, ddof = 1)
        seas_resid_var = np.var(resid_data+seas_data, ddof = 1)
        trend_strength = 1-resid_var/trend_resid_var
        seas_strength = 1-resid_var/seas_resid_var
        return trend_strength, seas_strength
    

In [8]:
def pred_stl(data_pred, period=None):
    '''
    Args:
        data_pred: prediction samples, ndarray, n_samples*pred_length or (pred_length,)
        period: Sometimes STL cannot recognize the period itself due to the length of the prediction steps
    Returns:
        average_pred_trend_strength: Value, float
        average_pred_seas_strength: Value, float
    '''
    if data_pred.ndim == 1:
        pred_stl = Stl_decomposition((data_pred,_), period=period)
        average_pred_trend_strength, average_pred_seas_strength = pred_stl.strength()
    else:
        iteration_num = data_pred.shape[0]
        pred_trend_strength_list = []
        pred_seas_strength_list = []  
        for i in range(iteration_num):
            pred_stl = Stl_decomposition((data_pred.iloc[i, :].values,_), period=period)
            pred_trend_strength, pred_seas_strength = pred_stl.strength()
            pred_trend_strength_list.append(pred_trend_strength)
            pred_seas_strength_list.append(pred_seas_strength)
        average_pred_trend_strength = np.average(np.array(pred_trend_strength_list))
        average_pred_seas_strength = np.average(np.array(pred_seas_strength_list))
    return average_pred_trend_strength, average_pred_seas_strength

In [9]:
def plot_preds(train, test, pred_dict, model_name, show_samples=False):
    pred = pred_dict['median']
    pred = pd.Series(pred, index=test.index)
    plt.figure(figsize=(8, 6), dpi=100)
    plt.plot(train)
    plt.plot(test, label='Truth', color='black')
    plt.plot(pred, label=model_name, color='purple')
    # shade 90% confidence interval
    samples = pred_dict['samples']
    lower = np.quantile(samples, 0.05, axis=0)
    upper = np.quantile(samples, 0.95, axis=0)
    plt.fill_between(pred.index, lower, upper, alpha=0.3, color='purple')
    if show_samples:
        samples = pred_dict['samples']
        # convert df to numpy array
        samples = samples.values if isinstance(samples, pd.DataFrame) else samples
        for i in range(min(10, samples.shape[0])):
            plt.plot(pred.index, samples[i], color='purple', alpha=0.3, linewidth=1)
    plt.legend(loc='upper left')
    if 'NLL/D' in pred_dict:
        nll = pred_dict['NLL/D']
        if nll is not None:
            plt.text(0.03, 0.85, f'NLL/D: {nll:.2f}', transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))
    plt.show()

In [10]:
def dataset_obtain(testfrac=0.2):
    '''
    获得 numpy 结构数据集
    '''
    datasets_path = {
    'us_births': 'datasets/monash/us_births.pkl',  # daily
    'saugeenday': 'datasets/monash/saugeenday.pkl'  # daily
    }
    datas = []
    for name, data_path in datasets_path.items():
        with open(data_path, 'rb') as file:
            # 加载数据
            data = pickle.load(file)
        series = pd.Series(np.concatenate((data[0][0][0], data[0][0][1])))
        splitpoint = int(len(series)*(1-testfrac))
        train = series.iloc[:splitpoint]  # Only test the last couples of samples
        test = series.iloc[splitpoint:]
        datas.append((train,test))
    return dict(zip(datasets_path.keys(),datas))

In [11]:
# 获得 period,仅考虑下面给定的数据集
def period_get(name):
    '''
    得到数据集的period
    Args: name: dataset name, string
    Returns: period: int
    '''
    period = 365
    if name == "HeartRateDataset":
        period = 144
    if name == "GasRateCO2Dataset":
        period = 39
    if name == "AirPassengersDataset":
        period = 12
    if name == "AusBeerDataset":
        period = 4
    if name == "MonthlyMilkDataset":
        period = 12
    if name == "SunspotsDataset":
        period = 12
    if name == "WineDataset":
        period = 12
    if name == "WoolyDataset":
        period = 4
    if name == "IstanbulTraffic":
        period = 24
    if name == "TurkeyPower":   # 手推出来的, 并非真实
        period = 7
    if name == "us_birth":   # 手推出来的, 并非真实
        period = 7
    if name == "saugeenday":   # 手推出来的, 并非真实
        period = 7
    return period

## Univariate Timeseries

### Data-preprocessing

In [12]:
def preprocessing_data(train, test):
    # 注意当传入地址时, 变量名复制, 而地址没有复制
    train_reshaped = train.values.reshape(-1, 1)
    test_reshaped = test.values.reshape(-1, 1)
    scaler = StandardScaler()
    scaler.fit(train_reshaped)
    standardized_train = scaler.transform(train_reshaped)
    train = pd.Series(standardized_train.flatten(), index=train.index)
    standardized_test = scaler.transform(test_reshaped)
    test = pd.Series(standardized_test.flatten(), index=test.index)    
    return train, test

In [13]:
datasets = get_datasets()  
datasets_tmp = get_memorization_datasets()
datasets.update(datasets_tmp)
datasets_tmp = dataset_obtain()
datasets.update(datasets_tmp)

for name in datasets.keys():
    train, test = datasets[name]
    train, test = preprocessing_data(train, test)
    dataset = (train, test)
    datasets[name] = dataset

d:\Desktop\Works_left_over\0-已完成 (形式上)\1-Time-LLM\2-llmtime-main\llmtime-main\data1\small_context.py:115: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  series = pd.read_csv(f, index_col=0, parse_dates=True).values.reshape(-1)


In [17]:
datasets['saugeenday'][0]

0       -0.464246
1       -0.482401
2       -0.407189
3       -0.407189
4       -0.389035
           ...   
18987   -0.256767
18988   -0.261954
18989   -0.267141
18990   -0.267141
18991   -0.267141
Length: 18992, dtype: float64

### Initialization

##### Originial

In [20]:
test_strength = {}  # 保存 test 的 strength

In [21]:
out_gpt_4 = {}
out_gpt_3 = {}
out_gemini = {}

In [22]:
pred_strength_avg_4 = {}
median_pred_strength_4 = {}
pred_strength_avg_3 = {}
median_pred_strength_3 = {}
pred_strength_avg_gemini = {}
median_pred_strength_gemini = {}

In [23]:
MAE_gpt_4 = []
MSE_gpt_4 = []
MAPE_gpt_4 = []
R_gpt_4 = []

In [24]:
MAE_gpt_3 = []
MSE_gpt_3 = []
MAPE_gpt_3 = []
R_gpt_3 = []

In [25]:
MAE_gemini = []
MSE_gemini = []
MAPE_gemini = []
R_gemini = []

##### with MA

In [26]:
out_gpt_4_MA = {}
out_gpt_3_MA = {}
out_gemini_MA = {}

In [27]:
MAE_gpt_4_MA = []
MSE_gpt_4_MA = []
MAPE_gpt_4_MA = []
R_gpt_4_MA = []

In [28]:
MAE_gpt_3_MA = []
MSE_gpt_3_MA = []
MAPE_gpt_3_MA = []
R_gpt_3_MA = []

In [29]:
MAE_gemini_MA = []
MSE_gemini_MA = []
MAPE_gemini_MA = []
R_gemini_MA = []

### Models Chosen

In [30]:
gpt4_hypers = dict(
    alpha=0.3,
    basic=True,
    temp=1.0,
    top_p=0.8,
    settings=SerializerSettings(base=10, prec=3, signed=True, time_sep=', ', bit_sep='', minus_sign='-')
)

gpt3_hypers = dict(
    temp=0.7,
    alpha=0.95,
    beta=0.3,
    basic=False,
    settings=SerializerSettings(base=10, prec=3, signed=True, half_bin_correction=True)
)

promptcast_hypers = dict(
    temp=0.7,
    settings=SerializerSettings(base=10, prec=0, signed=True,
                                time_sep=', ',
                                bit_sep='',
                                plus_sign='',
                                minus_sign='-',
                                half_bin_correction=False,
                                decimal_point='')
)

# arima_hypers = dict(p=[12, 30], d=[1, 2], q=[0])

gemini_hypers = {
    'temp': 0.2,  # 此处取 0.2 即可
    # temp 在0.2 or 0.4是比较好的，不过这个寻优只是通过 MAPE,R^2找到，因为gemini没办法得到输出token的log-p，无法调用validation-likelihood function
    # 话又说回来, 似乎validation-likelihood function也不一定使得模型表现更好......
    'alpha': 0.95,
    'beta': 0.3,
    'basic': [False],
    'settings': [SerializerSettings(base=10, prec=3, signed=True,half_bin_correction=True)],  # prec 取 3
}

In [31]:
model_hypers = {
    'LLMTime GPT-3.5': {'model': 'gpt-3.5-turbo-instruct', **gpt3_hypers},
    # 'LLMTime GPT-4': {'model': 'gpt-4', **gpt4_hypers},
    # 'LLMTime GPT-3': {'model': 'text-davinci-003', **gpt3_hypers},
    # 'PromptCast GPT-3': {'model': 'text-davinci-003', **promptcast_hypers},
    # 'ARIMA': arima_hypers,
    'gemini-1.0-pro': {'model': 'gemini-1.0-pro', **gemini_hypers},
}

model_predict_fns = {
    'LLMTime GPT-3.5': get_llmtime_predictions_data,
    # 'LLMTime GPT-4': get_llmtime_predictions_data,
    # 'PromptCast GPT-3': get_promptcast_predictions_data,
    # 'ARIMA': get_arima_predictions_data,
    'gemini-1.0-pro': get_llmtime_predictions_data,  # 此处假定用gpt系列的encoding方式来进行预测
}

In [32]:
model_names = list(model_predict_fns.keys())

### TS forecasting with pre-trained LLMs

### Datasets(Darts and Monash_Not commonly used datasets)

In [33]:
# Selected datasets
# name_list = ['GasRateCO2Dataset', 'WineDataset', 'SunspotsDataset', 'AusBeerDataset']

# only for partial test 
name_list = ['AirPassengersDataset', 'AusBeerDataset', 'GasRateCO2Dataset', 'MonthlyMilkDataset', 'SunspotsDataset', 'WineDataset', 'WoolyDataset', 'HeartRateDataset', 'IstanbulTraffic', 'TSMCStock', 'TurkeyPower', 'us_births', 'saugeenday']

In [34]:
def MA_process(name, train, test, test_length=96):
    period = period_get(name)
    data_tmp = pd.concat([train, test])
    out = data_tmp.rolling(window=period).mean().dropna()
    train_tmp = out[:-test_length]
    test_tmp = out[-test_length:]
    return train_tmp, test_tmp

def metrics_computation(test, median_pred):
    mse = mean_squared_error(test, median_pred)
    mae = mean_absolute_error(test, median_pred)
    mape = mean_absolute_percentage_error(test, median_pred) * 100
    r2 = r2_score(test, median_pred)        
    return mse, mae, mape, r2


def MA_recover(data_pred, median_pred, record_tmp, record_list_tmp, period=8, whether_MA=False):
    data_pred_tmp = copy.deepcopy(data_pred)
    median_pred_tmp = copy.deepcopy(median_pred)
    if whether_MA is True:
        for index in range(data_pred.shape[1]):  # 时间步
            if index >= period:
                data_pred.iloc[:, index] = period * (data_pred_tmp.iloc[:, index]-data_pred_tmp.iloc[:, index-1])+data_pred.iloc[:, index-period]
                median_pred.iloc[index] = period * (median_pred_tmp.iloc[index]-median_pred_tmp.iloc[index-1])+median_pred.iloc[index-period]
            else:
                if index == 0:
                    data_pred.iloc[:, index] = period * (data_pred_tmp.iloc[:, index]-record_tmp)+record_list_tmp.iloc[index]
                    median_pred.iloc[index] = period * (median_pred_tmp.iloc[index]-record_tmp)+record_list_tmp.iloc[index]  # Series 结构, 没有 iloc[:, 0], 只取.iloc[0]
                else:
                    data_pred.iloc[:, index] = period * (data_pred_tmp.iloc[:, index]-data_pred_tmp.iloc[:, index-1])+record_list_tmp.iloc[index]
                    median_pred.iloc[index] = period * (median_pred_tmp.iloc[index]-median_pred_tmp.iloc[index-1])+record_list_tmp.iloc[index]
    return data_pred, median_pred

def results_update(dataset_name, model_name, pred_dict, mae, mse, mape, r2, whether_MA=False):
    if not whether_MA:
        if model_name == 'LLMTime GPT-4':
            out_gpt_4.update({dataset_name: pred_dict})          
            MAE_gpt_4.append(mae)
            MSE_gpt_4.append(mse)
            MAPE_gpt_4.append(mape)
            R_gpt_4.append(r2)
            
            # pred_strength_avg_4.update({name:pred_stl(data_pred, period=period)})
            # median_pred_strength_4.update({name:pred_stl(median_pred, period=period)})
        if model_name == 'LLMTime GPT-3.5':
            out_gpt_3.update({dataset_name: pred_dict})            
            MAE_gpt_3.append(mae)
            MSE_gpt_3.append(mse)
            MAPE_gpt_3.append(mape)
            R_gpt_3.append(r2)
            
            # pred_strength_avg_3.update({name:pred_stl(data_pred, period=period)})
            # median_pred_strength_3.update({name:pred_stl(median_pred, period=period)})
        if model_name == 'gemini-1.0-pro':
            out_gemini.update({dataset_name: pred_dict})           
            MAE_gemini.append(mae)
            MSE_gemini.append(mse)
            MAPE_gemini.append(mape)
            R_gemini.append(r2)
    
            # pred_strength_avg_gemini.update({name:pred_stl(data_pred, period=period)})  # 恢复了
            # median_pred_strength_gemini.update({name:pred_stl(median_pred, period=period)})
    else:
        if model_name == 'LLMTime GPT-4':
            out_gpt_4_MA.update({dataset_name: pred_dict})          
            MAE_gpt_4_MA.append(mae)
            MSE_gpt_4_MA.append(mse)
            MAPE_gpt_4_MA.append(mape)
            R_gpt_4_MA.append(r2)
            
            # pred_strength_avg_4.update({name:pred_stl(data_pred, period=period)})
            # median_pred_strength_4.update({name:pred_stl(median_pred, period=period)})
        if model_name == 'LLMTime GPT-3.5':
            out_gpt_3_MA.update({dataset_name: pred_dict})            
            MAE_gpt_3_MA.append(mae)
            MSE_gpt_3_MA.append(mse)
            MAPE_gpt_3_MA.append(mape)
            R_gpt_3_MA.append(r2)
            
            # pred_strength_avg_3.update({name:pred_stl(data_pred, period=period)})
            # median_pred_strength_3.update({name:pred_stl(median_pred, period=period)})
        if model_name == 'gemini-1.0-pro':
            out_gemini_MA.update({dataset_name: pred_dict})           
            MAE_gemini_MA.append(mae)
            MSE_gemini_MA.append(mse)
            MAPE_gemini_MA.append(mape)
            R_gemini_MA.append(r2)
    
            # pred_strength_avg_gemini.update({name:pred_stl(data_pred, period=period)})  # 恢复了
            # median_pred_strength_gemini.update({name:pred_stl(median_pred, period=period)})       

In [35]:
# Hyperparameters chosen
whether_MA = False

In [36]:
def forecasting_w_LLMs(datasets, name_list, model_names, whether_MA=False, pred_period=8, whether_multi=False):
    
    for name, dataset in datasets.items():
        if name not in name_list:  # 仅仅用于测试
            continue
    
        data = datasets[name]
        train, test = data  # period 可以考虑为 365; train: 1024; test: 96
    
        if name in ['us_births', 'saugeenday']:  
            train = train.iloc[-3080:]  # 对于 gemini 的处理, 因为其没有 tokenize 的可直接调用的存在, 所以此处人为设置, 使得结果对齐
            test = test.iloc[:96]  # 避免超过最长 token 值 (gpt 都会更短) context length 包含 input 和 output length 之和; 对于 us_births 和 saugeenday 的处理, 使得 test set 只预测后面 96 个 numbers (对齐一般的 multi-variate 预测方法)
        
        test_original = copy.deepcopy(test)
        record_list_tmp = train[-pred_period:]
        
        if whether_MA is True:
            train, test = MA_process(name=name, train=train, test=test, test_length=len(test))
        record_tmp = train.iloc[-1]
    
        # test_strength.update({name:pred_stl(test, period=period)})
        for model in model_names: 
            print(model)
            if model == 'gemini-1.0-pro':
                whether_blank = False
            else:
                whether_blank = True
            model_hypers[model].update({'dataset_name': name})  # for promptcast, dict 添加元素
            hypers = list(grid_iter(model_hypers[model]))  # 简单理解为传参
            num_samples = 10  # 表示重复的预测的次数, 原本需要为10, 此处暂时选择 2 来进行测试
            
            print(type(train))
            pred_dict = get_autotuned_predictions_data(train, test, hypers, num_samples, model_predict_fns[model], verbose=False, parallel=False, whether_blanket=whether_blank, whether_multi=whether_multi) 
            data_pred = pred_dict['samples']  # Dataframe 结构, rows represent the num_samples and cols represent the forecasting steps
            median_pred = pred_dict['median'] 
            
            data_pred, median_pred = MA_recover(data_pred=data_pred, median_pred=median_pred, record_tmp=record_tmp, record_list_tmp=record_list_tmp, period=pred_period, whether_MA=whether_MA)
                            
            pred_dict['samples'] = data_pred  # 使得记录的预测结果也发生变化
            pred_dict['median'] = median_pred
    
            mse, mae, mape, r2 = metrics_computation(test_original, median_pred)
            # 目前只关注 gpt3.5 和 gpt 4
            results_update(name, model, pred_dict, mae, mse, mape, r2, whether_MA=whether_MA)

In [37]:
# whether_MA = False, training on the original methods

whether_MA = False
forecasting_w_LLMs(datasets, name_list, model_names, whether_MA=whether_MA)


LLMTime GPT-3.5
<class 'pandas.core.series.Series'>


KeyboardInterrupt: 

In [114]:
# whether_MA = False, training on the original methods

whether_MA = False
forecasting_w_LLMs(datasets, name_list, model_names, whether_MA=whether_MA)


LLMTime GPT-3.5
Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-3.5-turbo-instruct', 'temp': 0.7, 'alpha': 0.95, 'beta': 0.3, 'basic': False, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=' ,', bit_sep=' ', plus_sign='', minus_sign=' -', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'AirPassengersDataset'}) 
 with NLL inf



  0%|          | 0/1 [00:00<?, ?it/s]

 2 9 5 , 3 1 0 , 3 4 3 , 3 3 6 , 3 1 7 , 3 5 0 , 3 8 1 , 3 8 1 , 3 5 3 , 3 1 2 , 2 7 6 , 3 1 0 , 3 0 2 , 3 2 9 , 3 6 4 , 3 5 0 , 3 2 6 , 3 8 4 , 4 3 4 , 4 3 4 , 4 0 5 , 3 4 5 , 3 0 0 , 3 6 2 , 3 7 4 , 3 8 6 , 4 5 3 , 4 1 7 , 4 3 8 , 4 5 3 , 5 0 3 , 5 0 3 , 4 6 7 , 4 1 5 , 3 7 6 , 4 2 4 , 4 3 6 , 4 5 7 , 4 8 8 , 4 6 0 , 4 6 5 , 5 4 8 , 5 7 7 , 6 0 5 , 5 2 7 , 4 8 4 , 4 3 8 , 4 9 1 , 4 9 6 , 4 9 6 , 5 9 1 , 5 8 9 , 5 7 4 , 6 0 8 , 6 5 8 , 6 7 7 , 5 9 3 , 5 3 1 , 4 5 7 , 5 0 7 , 5 1 5 , 4 7 6 , 5 8 9 , 5 6 9 , 5 8 6 , 6 5 8 , 7 4 8 , 7 2 7 , 6 4 6 , 5 7 4 , 5 1 2 , 5 7 4 , 6 0 5 , 5 8 4 , 6 6 5 , 6 7 0 , 6 7 2 , 7 7 9 , 8 9 6 , 8 5 6 , 7 7 2 , 6 8 2 , 5 9 3 , 6 9 1 , 7 0 5 , 6 8 9 , 7 8 4 , 7 7 4 , 7 8 6 , 9 2 0 , 1 0 1 3 , 9 9 4 , 8 7 5 , 7 5 8 , 6 7 4 , 7 5 8 , 7 7 9 , 7 4 6 , 8 7 7 , 8 5 8 , 8 7 5 , 1 0 3 4 , 1 1 3 7 , 1 1 4 2 , 9 9 1 , 8 5 6 , 7 5 5 , 8 2 9 , 8 3 9 , 7 8 6 , 8 9 1 , 8 5 8 , 8 9 4 , 1 0 6 5 , 1 1 9 9 ,



100%|██████████| 1/1 [00:06<00:00,  6.04s/it]


LLMTime GPT-3.5
Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-3.5-turbo-instruct', 'temp': 0.7, 'alpha': 0.95, 'beta': 0.3, 'basic': False, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=' ,', bit_sep=' ', plus_sign='', minus_sign=' -', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'AusBeerDataset'}) 
 with NLL inf


...


100%|██████████| 1/1 [00:05<00:00,  5.87s/it]


LLMTime GPT-3.5
Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-3.5-turbo-instruct', 'temp': 0.7, 'alpha': 0.95, 'beta': 0.3, 'basic': False, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=' ,', bit_sep=' ', plus_sign='', minus_sign=' -', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'us_births'}) 
 with NLL inf



  0%|          | 0/1 [00:00<?, ?it/s]

 8 3 5 ,


InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 6143 tokens (7 in your prompt; 6136 for the completion). Please reduce your prompt; or completion length.

### 临时处理 us_births 和 'sa    ugeenday'

In [172]:
datasets_check = dataset_obtain()

for name in datasets_check.keys():
    train, test = datasets_check[name]
    train, test = preprocessing_data(train, test)
    dataset = (train, test)
    datasets_check[name] = dataset

In [174]:
# 临时测试
# whether_MA = False, training on the original methods

whether_MA = False
forecasting_w_LLMs(datasets_check, name_list, model_names, whether_MA=whether_MA)


LLMTime GPT-3.5
Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-3.5-turbo-instruct', 'temp': 0.7, 'alpha': 0.95, 'beta': 0.3, 'basic': False, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=' ,', bit_sep=' ', plus_sign='', minus_sign=' -', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'us_births'}) 
 with NLL inf


  0%|          | 0/1 [00:00<?, ?it/s]

 8 3 4 , 9 0 3 , 9 0 1 , 8 4 3 , 8 9 5 , 5 2 9 , 5 1 7 , 8 3 9 , 8 7 2 , 8 2 5 , 8 5 6 , 8 2 0 , 5 3 9 , 4 6 9 , 8 2 6 , 8 9 7 , 8 2 1 , 8 3 2 , 8 4 0 , 5 1 7 , 5 1 6 , 7 2 8 , 8 3 5 , 8 0 0 , 8 3 1 , 8 3 3 , 5 4 2 , 4 6 6 , 8 2 8 , 8 6 8 , 8 1 8 , 8 3 3 , 8 0 5 , 5 0 4 , 5 0 7 , 8 2 5 , 9 0 8 , 8 3 1 , 8 2 5 , 8 7 2 , 5 4 9 , 4 8 4 , 8 7 1 , 9 4 1 , 8 4 6 , 4 8 3 , 6 8 7 , 4 6 9 , 4 6 7 , 8 1 7 , 8 8 0 , 7 7 1 , 8 3 9 , 7 9 8 , 4 7 6 , 4 2 2 , 8 1 7 , 8 3 5 , 8 0 9 , 8 1 0 , 8 0 9 , 4 9 8 , 4 8 3 , 8 3 2 , 8 6 2 , 8 3 4 , 8 6 8 , 8 8 1 , 4 8 0 , 4 5 7 , 9 2 2 , 9 6 2 , 8 2 6 , 6 9 7 , 6 1 2 , 4 3 4 , 3 6 3 , 4 6 9 , 8 9 6 , 1 0 1 2 , 9 4 4 , 9 6 5 , 5 2 8 , 4 2 9 , 4 2 8 , 7 2 5 , 7 7 1 , 7 8 6 , 8 0 6 , 5 1 6 , 4 6 8 , 7 3 0 , 8 0 6 , 7 5 2 , 7 4 9 , 7 5 6 , 5 1 7 , 4 4 6 , 7 7 4 , 8 0 2 , 8 0 9 , 7 6 6 , 8 0 3 , 4 8 7 , 4 4 0 , 7 7 2 , 8 3 6 , 8 3 1 , 8 1 1 , 8 3 7 , 5 2 5 , 4 6 1 , 7 6 4 , 8 0 8 , 7 9 2 , 8 0 8 , 8 2 1 , 5 1 9 , 4 7 6 , 7 7 7 , 8 0 0 , 8 0 8 , 7 8 5 , 8 9 0 , 5 6 1

100%|██████████| 1/1 [00:14<00:00, 14.27s/it]


LLMTime GPT-3.5
Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-3.5-turbo-instruct', 'temp': 0.7, 'alpha': 0.95, 'beta': 0.3, 'basic': False, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=' ,', bit_sep=' ', plus_sign='', minus_sign=' -', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'saugeenday'}) 
 with NLL inf


  0%|          | 0/1 [00:00<?, ?it/s]

 6 5 7 , 6 5 6 , 6 6 6 , 6 8 2 , 7 0 5 , 7 0 0 , 6 8 8 , 6 7 6 , 6 6 7 , 6 6 7 , 7 0 6 , 7 5 0 , 7 6 1 , 7 5 1 , 7 3 9 , 7 5 1 , 7 5 4 , 7 4 6 , 7 3 7 , 7 2 7 , 7 1 3 , 7 0 1 , 6 9 2 , 6 9 0 , 7 2 7 , 7 7 2 , 8 0 6 , 7 9 7 , 7 7 0 , 7 5 0 , 7 3 8 , 7 2 4 , 7 1 4 , 7 1 2 , 7 3 3 , 7 6 7 , 7 6 7 , 7 5 7 , 7 4 0 , 7 2 4 , 7 1 3 , 7 1 5 , 7 5 3 , 7 7 5 , 7 6 0 , 7 4 2 , 7 3 0 , 7 2 4 , 7 2 0 , 7 4 4 , 8 4 5 , 8 7 9 , 8 4 8 , 8 0 9 , 7 9 3 , 8 1 3 , 8 3 5 , 8 8 5 , 8 9 5 , 8 7 9 , 1 0 0 0 , 1 0 8 7 , 1 0 0 7 , 9 0 2 , 8 5 3 , 8 1 5 , 8 1 2 , 8 7 2 , 8 7 5 , 8 5 6 , 8 0 5 , 7 8 9 , 7 8 2 , 7 7 3 , 7 7 2 , 8 0 0 , 9 6 7 , 1 1 8 3 , 1 2 3 4 , 1 0 8 2 , 9 3 5 , 8 6 2 , 8 1 8 , 7 6 5 , 7 8 0 , 8 1 0 , 7 7 7 , 9 0 6 , 1 2 0 9 , 1 1 0 7 , 1 0 3 7 , 1 0 3 5 , 9 6 7 , 9 1 5 , 1 0 5 8 , 1 3 0 5 , 1 2 1 4 , 1 0 4 1 , 9 0 2 , 8 3 9 , 8 0 8 , 7 7 9 , 7 2 8 , 7 2 6 , 7 2 1 , 7 1 7 , 7 1 4 , 7 1 3 , 7 1 0 , 7 0 8 , 7 0 7 , 7 0 4 , 7 0 1 , 7 0 0 , 6 9 9 , 6 9 7 , 6 9 6 , 6 9 4 , 6 9 3 , 6 9 1 , 6 9 1 , 6 9

100%|██████████| 1/1 [00:13<00:00, 13.80s/it]


In [175]:
print('Over!!')

Over!!


In [223]:
metrics_models = pd.Series(['MSE_gpt_3', 'MAE_gpt_3', 'MAPE_gpt_3', 'R_gpt_3', 'MSE_gemini', 'MAE_gemini', 'MAPE_gemini', 'R_gemini'])
tmp = pd.DataFrame([MSE_gpt_3, MAE_gpt_3, MAPE_gpt_3, R_gpt_3, MSE_gemini, MAE_gemini, MAPE_gemini, R_gemini], columns=datasets.keys())
pd.concat((metrics_models, tmp), axis=1).to_csv('outputs_updated/output_without_MA.csv')

In [ ]:
# WoolyDataset 很合适 (可以作为长程预测的方法)

train, test = datasets['WoolyDataset']
plot_preds(train=train, test=test, pred_dict=out_gemini['WoolyDataset'], model_name='LLMTime-gemini_0603')

### with MA_results

In [242]:
whether_MA = True
forecasting_w_LLMs(datasets, name_list, model_names, whether_MA=whether_MA)

LLMTime GPT-3.5
Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-3.5-turbo-instruct', 'temp': 0.7, 'alpha': 0.95, 'beta': 0.3, 'basic': False, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=' ,', bit_sep=' ', plus_sign='', minus_sign=' -', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'AirPassengersDataset'}) 
 with NLL inf


  0%|          | 0/1 [00:00<?, ?it/s]

 2 3 4 , 2 3 4 , 2 3 6 , 2 3 9 , 2 4 0 , 2 4 1 , 2 4 5 , 2 5 1 , 2 5 7 , 2 6 2 , 2 6 6 , 2 6 9 , 2 7 4 , 2 8 2 , 2 8 8 , 2 9 8 , 3 0 5 , 3 1 7 , 3 2 5 , 3 3 3 , 3 4 0 , 3 4 7 , 3 5 4 , 3 6 3 , 3 7 0 , 3 7 6 , 3 8 4 , 3 8 8 , 3 9 3 , 3 9 6 , 4 0 6 , 4 1 4 , 4 2 5 , 4 3 2 , 4 3 9 , 4 4 6 , 4 5 3 , 4 6 0 , 4 6 4 , 4 7 5 , 4 8 9 , 5 0 1 , 5 0 8 , 5 1 7 , 5 2 5 , 5 3 2 , 5 3 7 , 5 3 9 , 5 4 1 , 5 4 3 , 5 4 1 , 5 4 1 , 5 3 9 , 5 4 0 , 5 4 5 , 5 5 5 , 5 6 1 , 5 6 6 , 5 7 1 , 5 7 7 , 5 8 4 , 5 9 4 , 6 0 6 , 6 1 4 , 6 2 5 , 6 3 5 , 6 4 8 , 6 6 4 , 6 7 8 , 6 9 2 , 7 0 4 , 7 1 3 , 7 2 5 , 7 3 6 , 7 4 8 , 7 6 1 , 7 7 2 , 7 8 5 , 8 0 0 , 8 1 3 , 8 2 8 , 8 3 9 , 8 4 8 , 8 5 6 , 8 6 4 , 8 7 2 , 8 7 8 , 8 8 8 , 8 9 7 , 9 0 7 , 9 1 9 , 9 3 3 , 9 4 9 , 9 6 2 , 9 7 3 , 9 8 1 , 9 8 9 , 9 9 6 , 1 0 0 0 , 1 0 0 2 , 1 0 0 2 , 1 0 0 4 , 1 0 0 7 , 1 0 1 4 ,


100%|██████████| 1/1 [00:06<00:00,  6.14s/it]


gemini-1.0-pro
Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gemini-1.0-pro', 'temp': 0.2, 'alpha': 0.95, 'beta': 0.3, 'basic': False, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=' ,', bit_sep=' ', plus_sign='', minus_sign=' -', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'AirPassengersDataset'}) 
 with NLL inf


...

  0%|          | 0/1 [00:00<?, ?it/s]

745,767,806,853,898,939,975,1006,1028,1028,1024,1020,998,950,895,836,779,726,679,641,645,667,685,693,706,717,717,708,708,717,734,755,766,758,740,724,718,724,724,724,724,724,724,725,723,726,729,733,733,733,733,736,738,743,745,744,736,737,736,733,722,717,717,717,717,718,718,722,723,717,713,711,711,713,714,714,714,710,707,701,700,700,702,706,708,708,707,707,707,706,706,708,710,710,712,712,712,710,710,712,718,725,731,729,727,725,725,725,725,772,815,854,893,933,963,991,1002,1010,1023,1037,1033,1014,991,962,920,866,802,747,710,690,673,658,654,620,600,592,592,603,628,638,634,625,610,598,596,600,608,622,633,637,637,637,637,637,637,637,634,622,599,569,534,492,450,409,369,323,278,256,243,239,242,242,253,280,318,361,414,452,469,482,489,493,493,492,488,479,467,452,448,478,527,559,584,604,617,627,633,642,650,661,667,678,700,710,718,721,724,725,732,741,747,756,767,
Index: 0
['7 7 7 , 7 8 4 , 7 8 9 , 7 9 0 , 7 8 9 , 7 8 9 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0

100%|██████████| 1/1 [09:23<00:00, 563.42s/it]

['7 7 7 , 7 8 4 , 7 8 9 , 7 9 0 , 7 8 9 , 7 8 9 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 , 7 9 0 

IndexError: single positional indexer is out-of-bounds

In [ ]:
print("over!!!")

In [218]:
train_strength = {}
for period in [5, 7, 15, 30, 60, 90, 120, 180, 360]:
    print({'us_birth':pred_stl(datasets['us_births'][0], period=period)})
    print({'saugeenday':pred_stl(datasets['saugeenday'][0], period=period)})


{'us_birth': (0.5639612553686942, 0.010866728632411227)}
{'saugeenday': (0.9428505253333315, 0.2778664387228216)}
{'us_birth': (0.924711678853785, 0.9239760424454276)}
{'saugeenday': (0.9122881440783992, 0.3093161120750595)}
{'us_birth': (0.6408114192682991, 0.5018234370058727)}
{'saugeenday': (0.8248051955780521, 0.3257435096630418)}
{'us_birth': (0.4734136915371586, 0.040224494450762927)}
{'saugeenday': (0.7091386646774228, 0.33681297964617196)}
{'us_birth': (0.5164370112744445, 0.22523686221290673)}
{'saugeenday': (0.5738881747705757, 0.35015940840514415)}
{'us_birth': (0.6069529194527568, 0.4854551536045587)}
{'saugeenday': (0.4719871133850344, 0.3381557805208196)}
{'us_birth': (0.5891568212299283, 0.4733984079166731)}
{'saugeenday': (0.40785939780381486, 0.34476218442026385)}
{'us_birth': (0.42971867521115226, 0.11929315926101802)}
{'saugeenday': (0.31051563644548597, 0.3746492226938203)}
{'us_birth': (0.44877497933842214, 0.38367871021812994)}
{'saugeenday': (0.1112369030809115, 

# Multi-variate datasets

In [82]:
def get_datasets_multi(n=-1, testfrac=0.2):
    datasets = [
        'electricity',
        'ETTh1',  
        'ETTh2',
        'ETTm1',
        'ETTm2', 
        'exchange_rate',
        'national_illness',
        'traffic',
        'weather'
    ]
    datas = []
    for i, dsname in enumerate(datasets):
        path = f'datasets/multi_variate_datasets/{dsname}.csv'
        series = pd.read_csv(path)
        splitpoint = int(series.shape[0]*(1-testfrac))
        train = series.iloc[:splitpoint]  # Only test the last couples of samples
        test = series.iloc[splitpoint:]
        datas.append((train,test))
        if i+1==n:
            break
    return dict(zip(datasets,datas))

In [83]:
datasets_tmp = get_datasets_multi()

#### 使用单变量进行预测

In [124]:
for name, dataset in datasets_tmp.items():
    def metrics_computation(test, median_pred):
        mse = mean_squared_error(test, median_pred)
        mae = mean_absolute_error(test, median_pred)
        mape = mean_absolute_percentage_error(test, median_pred) * 100
        r2 = r2_score(test, median_pred)        
        return mse, mae, mape, r2
        
    train, test = dataset  # period 可以考虑为 365
    train = train[len(train)-1024:]['OT']  # train set 太长了，这里统一取 1024 个数字进行预测
    test = test[:96]['OT']  # 这里预测后 96 个数字 (因为 test set 太长了，完全预测消耗太多)
    
    test_strength.update({name:pred_stl(test, period=365)})
    for model in model_names: 
        model_hypers[model].update({'dataset_name': name})  # for promptcast, dict 添加元素
        hypers = list(grid_iter(model_hypers[model]))  # 简单理解为传参
        num_samples = 10  # 表示重复的预测的次数, 原本需要为10, 此处暂时选择 2 来进行测试
        pred_dict = get_autotuned_predictions_data(train, test, hypers, num_samples, model_predict_fns[model], verbose=False, parallel=False, whether_blanket=False) 
        # 目前只关注 gpt3.5 和 gpt 4
        if model == 'LLMTime GPT-4':
            out_gpt_4.update({name: pred_dict})
            data_pred = pred_dict['samples']
            median_pred = pred_dict['median']  # 通过计算 median 来计算相关指标
            
            mse, mae, mape, r2 = metrics_computation(test, median_pred)
            MAE_gpt_4.append(mae)
            MSE_gpt_4.append(mse)
            MAPE_gpt_4.append(mape)
            R_gpt_4.append(r2)
            
            pred_strength_avg_4.update({name:pred_stl(data_pred, period=365)})
            median_pred_strength_4.update({name:pred_stl(median_pred, period=365)})
        if model == 'LLMTime GPT-3.5':
            out_gpt_3.update({name: pred_dict})
            data_pred = pred_dict['samples']
            median_pred = pred_dict['median']
            
            mse, mae, mape, r2 = metrics_computation(test, median_pred)
            MAE_gpt_3.append(mae)
            MSE_gpt_3.append(mse)
            MAPE_gpt_3.append(mape)
            R_gpt_3.append(r2)
            
            pred_strength_avg_3.update({name:pred_stl(data_pred, period=365)})
            median_pred_strength_3.update({name:pred_stl(median_pred, period=365)})
        if model == 'gemini-1.0-pro':
            out_gemini.update({name: pred_dict})
            data_pred = pred_dict['samples']
            median_pred = pred_dict['median']
            
            mse, mae, mape, r2 = metrics_computation(test, median_pred)
            MAE_gemini.append(mae)
            MSE_gemini.append(mse)
            MAPE_gemini.append(mape)
            R_gemini.append(r2)

            pred_strength_avg_gemini.update({name:pred_stl(data_pred, period=365)})
            median_pred_strength_gemini.update({name:pred_stl(median_pred, period=365)})


Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-4', 'alpha': 0.3, 'basic': True, 'temp': 1.0, 'top_p': 0.8, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=', ', bit_sep='', plus_sign='', minus_sign='-', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'electricity'}) 
 with NLL inf




  0%|          | 0/1 [00:00<?, ?it/s]

943, 951, 970, 1018, 1075, 1171, 1250, 1226, 1231, 1307, 1274, 1313, 1410, 1450, 1501, 1421, 1306, 1284, 1204, 1151, 1121, 1086, 1063, 1037, 1016, 996, 1020, 961, 1030, 1123, 1186, 1198, 1320, 1337, 1376, 1384, 1369, 1408, 1459, 1385, 1334, 1301, 1141, 1167, 1109, 1025, 1049, 1009, 1007, 975, 1003, 936, 1046, 1158, 1180, 1237, 1266, 1307, 1305, 1361, 1457, 1528, 1488, 1324, 1259, 1261, 1181, 1154, 1136, 1101, 1048, 1055, 1023, 993, 970, 975, 1059, 1163, 1204, 1256, 1329, 1353, 1377, 1542, 1575, 1661, 1772, 1610, 1512, 1364, 1242, 1182, 1147, 1130, 1105, 1069, 1051, 1046, 1049, 1038, 1078, 1131, 1151, 1191, 1203, 1156, 1150, 1179, 1215, 1242, 1216, 1249, 1239, 1176, 1124, 1119, 1077, 1035, 1045, 1029, 1023, 1015, 982, 998, 936, 986, 964, 1031, 1034, 1010, 1024, 1068, 1036, 1080, 1050, 1021, 987, 999, 951, 973, 920, 937, 887, 923, 874, 915, 898, 913, 734, 1013, 952, 937, 992, 988, 944, 1005, 957, 990, 953, 950, 970, 945, 912, 950, 884, 909, 912, 899, 887, 890, 890, 972, 985, 1095, 1112, 



100%|██████████| 1/1 [00:17<00:00, 17.81s/it]


Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-4', 'alpha': 0.3, 'basic': True, 'temp': 1.0, 'top_p': 0.8, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=', ', bit_sep='', plus_sign='', minus_sign='-', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'ETTh1'}) 
 with NLL inf


...



100%|██████████| 1/1 [00:20<00:00, 20.05s/it]


Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-4', 'alpha': 0.3, 'basic': True, 'temp': 1.0, 'top_p': 0.8, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=', ', bit_sep='', plus_sign='', minus_sign='-', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'weather'}) 
 with NLL inf




  0%|          | 0/1 [00:00<?, ?it/s]

987, 986, 984, 985, 987, 987, 987, 986, 996, 1001, 996, 986, 990, 1008, 1015, 1006, 1010, 1007, 1005, 999, 1004, 1032, 1017, 1008, 1010, 1012, 1014, 1023, 1019, 1016, 1028, 1025, 1034, 1033, 1039, 1036, 1034, 1044, 1055, 1064, 1064, 1074, 1075, 1070, 1072, 1067, 1059, 1053, 1057, 1054, 1059, 1080, 1086, 1080, 1077, 1085, 1075, 1078, 1066, 1068, 1073, 1072, 1079, 1082, 1081, 1074, 1072, 1075, 1074, 1075, 1092, 1080, 1078, 1081, 1089, 1099, 1097, 1101, 1099, 1094, 1092, 1086, 1087, 1093, 1084, 1080, 1092, 1099, 1093, 1107, 1103, 1113, 1114, 1108, 1116, 1120, 1124, 1113, 1115, 1120, 1120, 1114, 1095, 1086, 1045, 1027, 1034, 1037, 1035, 1025, 1018, 1020, 1018, 1013, 1013, 1011, 1001, 1001, 1005, 1006, 1007, 1005, 1005, 1000, 998, 999, 996, 996, 996, 997, 1006, 1006, 1006, 1007, 1007, 1005, 1006, 1002, 1001, 998, 993, 988, 989, 990, 990, 993, 996, 997, 997, 993, 990, 991, 993, 992, 992, 994, 1001, 998, 998, 999, 1008, 1016, 1004, 1001, 1000, 999, 1000, 1003, 1005, 1005, 1005, 1005, 1007, 10



100%|██████████| 1/1 [00:19<00:00, 19.21s/it]


#### multi-variate time series forecasting

In [ ]:
out_gpt_4_multi = {}
pred_strength_avg_4_multi = {}
median_pred_strength_4_multi = {}
MAE_gpt_4_multi = []
MSE_gpt_4_multi = []
MAPE_gpt_4_multi = []
R_gpt_4_multi = []

out_gpt_3_multi = {}
pred_strength_avg_3_multi = {}
median_pred_strength_3_multi = {}
MAE_gpt_3_multi = []
MSE_gpt_3_multi = []
MAPE_gpt_3_multi = []
R_gpt_3_multi = []

out_gemini_multi = {}
pred_strength_avg_gemini_multi = {}
median_pred_strength_gemini_multi = {}
MAE_gemini_multi = []
MSE_gemini_multi = []
MAPE_gemini_multi = []
R_gemini_multi = []

In [148]:
np.random.seed(42)

random_selection = np.random.choice(np.arange(10), size=5, replace=False)
random_selection

array([8, 1, 5, 0, 7])

In [149]:
train, test = dataset
test.iloc[:, random_selection]

,VPdef (mbar),p (mbar),rh (%),date,VPact (mbar)
42156,2.22,995.43,78.13,2020-10-19 19:30:00,7.93
42157,2.21,995.36,78.07,2020-10-19 19:40:00,7.85
42158,2.24,995.28,77.76,2020-10-19 19:50:00,7.82
42159,2.16,995.38,78.40,2020-10-19 20:00:00,7.84
42160,2.09,995.41,78.87,2020-10-19 20:10:00,7.81
...,...,...,...,...,...
52691,1.44,978.32,80.00,2020-12-31 23:20:00,5.76
52692,1.20,978.30,83.10,2020-12-31 23:30:00,5.92
52693,1.26,978.26,82.20,2020-12-31 23:40:00,5.80
52694,1.32,978.26,81.40,2020-12-31 23:50:00,5.77


In [161]:
for index, name in enumerate(out_gpt_3.keys()):
    print(f'{name}_MAE:', MAE_gpt_3[index])
    print(f'{name}_MSE:', MSE_gpt_3[index])

us_births_MAE: 904.8080234375002
us_births_MSE: 1323052.464514876
saugeenday_MAE: 26.15595005615954
saugeenday_MSE: 4100.271746224603
electricity_MAE: 561.8401705729169
electricity_MSE: 513239.62302456633
ETTh1_MAE: 1.1488348688690617
ETTh1_MSE: 2.2016950483083413
ETTh2_MAE: 6.649757592859662
ETTh2_MSE: 65.397631525492
ETTm1_MAE: 1.6014383428176249
ETTm1_MSE: 3.1490036781487016
ETTm2_MAE: 5.088133394259949
ETTm2_MSE: 33.984473732095246
exchange_rate_MAE: 0.01674936216979168
exchange_rate_MSE: 0.0004495669140553021
national_illness_MAE: 368425.46821770835
national_illness_MSE: 179785564832.8517
traffic_MAE: 0.012021336458333337
traffic_MSE: 0.00022579304360938166
weather_MAE: 12.339366250000005
weather_MSE: 203.03844172831273
